In [1]:
from google.colab import drive
drive.mount("/content/drive")
import keras
from keras.optimizers import Adam
import scipy
import tensorflow as tf
from keras import layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    AveragePooling2D,
    BatchNormalization,
    Conv2D,
    Dense,
    Flatten,
    MaxPool2D,
    PReLU,
    ReLU,
    MaxPooling2D,
    Add
)
from keras.callbacks import ModelCheckpoint
from keras import regularizers

from sklearn.model_selection import KFold
import numpy as np


Mounted at /content/drive


In [2]:
import os
import cv2


def load_images(fold):
  directory = rf"/content/drive/MyDrive/MLProject/dataset/{fold}"

  num_images = sum([len(files) for r, d, files in os.walk(directory)])
  print(num_images)
  i = 0
  #imgs = []
  imgs = np.empty((num_images, 224, 224, 3), dtype=np.uint8)
  #lbls = np.empty(num_images)
  labels = []
  for class_name in os.listdir(directory):
    class_dir = os.path.join(directory, class_name)
    print(class_dir)
    for file in os.listdir(class_dir):
      img_path = os.path.join(class_dir, file)
      img = cv2.imread(img_path)
      img = cv2.resize(img, (224, 224))
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      imgs[i] = img
      #imgs.append(img)
      labels.append(class_name)
      #lbls[i] = class_dir
      i += 1

  print('returning')
  return imgs, np.array(labels)



In [3]:
train_img, train_labels = load_images('train')
test_img, test_labels = load_images('test')

4733
/content/drive/MyDrive/MLProject/dataset/train/muffin
/content/drive/MyDrive/MLProject/dataset/train/chihuahua
returning
1184
/content/drive/MyDrive/MLProject/dataset/test/chihuahua
/content/drive/MyDrive/MLProject/dataset/test/muffin
returning


In [4]:
inputs = np.concatenate((train_img, test_img), axis=0)
targets = np.concatenate((train_labels, test_labels), axis=0)

In [5]:
kfold = KFold(n_splits=5, shuffle=True)

In [6]:


# Encode target labels


In [7]:
one_zero_losses = []
fold = 1
from sklearn.preprocessing import LabelEncoder

for train_index, test_index in kfold.split(inputs):

  train_X = inputs[train_index]
  val_X = inputs[test_index]
  train_y  = targets[train_index]
  val_y = targets[test_index]

  label_encoder = LabelEncoder()
  train_y_encoded = label_encoder.fit_transform(train_y)
  val_y_encoded = label_encoder.transform(val_y)

  checkpoint = ModelCheckpoint(filepath='/content/dirve/MyDrive/MLProject/models/sequential_v02/best_model_cv.keras',
                              monitor='val_accuracy',
                              mode='max',
                              save_best_only=True,
                              verbose = 1)

  # import the model
  seq = tf.keras.models.load_model("/content/drive/MyDrive/MLProject/models/sequential_v02/seq_v02.keras", safe_mode = False)
  seq.compile(loss="binary_crossentropy",
              optimizer= Adam(learning_rate=0.0001),
              metrics=["accuracy"])

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold} ...')

  steps_per_epoch = train_X.shape[0] / 32

  history = seq.fit(train_X, train_y_encoded, epochs=50, batch_size = 32, steps_per_epoch = steps_per_epoch, callbacks = [checkpoint],
                    validation_data=(val_X, val_y_encoded))

  best_model = tf.keras.models.load_model("/content/dirve/MyDrive/MLProject/models/sequential_v02/best_model_cv.keras", safe_mode = False)
  predictions = best_model.predict(val_X)
  binary_predictions = (predictions >= 0.5).astype(int)
  binary_predictions = np.squeeze(binary_predictions)

  loss = sum(binary_predictions != val_y_encoded)/len(val_y_encoded)

  one_zero_losses.append(loss)


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
148/147 [==============================] - ETA: 0s - loss: 0.7900 - accuracy: 0.7589
Epoch 1: val_accuracy improved from -inf to 0.70524, saving model to /content/dirve/MyDrive/MLProject/models/sequential_v02/best_model_cv.keras
147/147 [==============================] - 27s 103ms/step - loss: 0.7900 - accuracy: 0.7589 - val_loss: 0.6933 - val_accuracy: 0.7052
Epoch 2/50
148/147 [==============================] - ETA: 0s - loss: 0.4748 - accuracy: 0.8301
Epoch 2: val_accuracy improved from 0.70524 to 0.83108, saving model to /content/dirve/MyDrive/MLProject/models/sequential_v02/best_model_cv.keras
147/147 [==============================] - 13s 88ms/step - loss: 0.4748 - accuracy: 0.8301 - val_loss: 0.4726 - val_accuracy: 0.8311
Epoch 3/50
147/147 [============================>.] - ETA: 0s - loss: 0.4576 - accuracy: 0.8355
Epoch 3: val_accuracy did not improve from 0.83108
147/14

In [8]:
one_zero_losses

[0.08614864864864864,
 0.06672297297297297,
 0.055790363482671176,
 0.07016060862214708,
 0.073541842772612]

In [7]:
one_zero_losses

[0.06503378378378379,
 0.08699324324324324,
 0.07945900253592561,
 0.07776838546069316,
 0.06677937447168217]

In [10]:
%run /content/drive/MyDrive/MLProject/code/residual_learning.py

In [6]:
one_zero_losses_lr = []
fold = 1
from sklearn.preprocessing import LabelEncoder

for train_index, test_index in kfold.split(inputs):

  train_X = inputs[train_index]
  val_X = inputs[test_index]
  train_y  = targets[train_index]
  val_y = targets[test_index]

  label_encoder = LabelEncoder()
  train_y_encoded = label_encoder.fit_transform(train_y)
  val_y_encoded = label_encoder.transform(val_y)

  checkpoint = ModelCheckpoint(filepath='/content/dirve/MyDrive/MLProject/models/residual_learning/best_model_cv.keras',
                              monitor='val_accuracy',
                              mode='max',
                              save_best_only=True,
                              verbose = 1)

  # import the model
  seq = tf.keras.models.load_model("/content/drive/MyDrive/MLProject/models/residual_learning/residual_learning.keras", safe_mode = False)
  seq.compile(loss="binary_crossentropy",
              optimizer= Adam(learning_rate=0.0001),
              metrics=["accuracy"])

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold} ...')

  steps_per_epoch = train_X.shape[0] / 32

  history = seq.fit(train_X, train_y_encoded, epochs=50, batch_size = 32, steps_per_epoch = steps_per_epoch, callbacks = [checkpoint],
                    validation_data=(val_X, val_y_encoded))

  best_model = tf.keras.models.load_model('/content/dirve/MyDrive/MLProject/models/residual_learning/best_model_cv.keras', safe_mode = False)
  predictions = best_model.predict(val_X)
  binary_predictions = (predictions >= 0.5).astype(int)
  binary_predictions = np.squeeze(binary_predictions)

  loss = sum(binary_predictions != val_y_encoded)/len(val_y_encoded)

  one_zero_losses_lr.append(loss)
  fold += 1



------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
148/147 [==============================] - ETA: 0s - loss: 0.7047 - accuracy: 0.7205
Epoch 1: val_accuracy improved from -inf to 0.62584, saving model to /content/dirve/MyDrive/MLProject/models/residual_learning/best_model_cv.keras
147/147 [==============================] - 32s 122ms/step - loss: 0.7047 - accuracy: 0.7205 - val_loss: 0.8646 - val_accuracy: 0.6258
Epoch 2/50
148/147 [==============================] - ETA: 0s - loss: 0.4814 - accuracy: 0.8136
Epoch 2: val_accuracy improved from 0.62584 to 0.84206, saving model to /content/dirve/MyDrive/MLProject/models/residual_learning/best_model_cv.keras
147/147 [==============================] - 13s 91ms/step - loss: 0.4814 - accuracy: 0.8136 - val_loss: 0.3592 - val_accuracy: 0.8421
Epoch 3/50
148/147 [==============================] - ETA: 0s - loss: 0.4411 - accuracy: 0.8253
Epoch 3: val_accuracy did not improve from 0.84206


In [7]:
one_zero_losses_lr

[0.08192567567567567,
 0.07516891891891891,
 0.07692307692307693,
 0.08030431107354184,
 0.09805579036348266]